In [1]:
# !pip install ollama jupyter -U

In [2]:
import requests
import json
import ollama
from IPython import display
from typing import List, Dict

**Documentation**:
https://github.com/ollama/ollama/tree/main/docs

In [3]:
text = """
Meta thông báo mở rộng công cụ Meta AI, hoạt động tương tự ChatGPT, đến 21 quốc gia mới, trong đó có thị trường Việt Nam.
Ngày 9/10, công ty mẹ của Facebook cho biết bộ tính năng Meta AI đã có mặt tại Brazil, Bolivia, Guatemala, Paraguay, Philippines và Anh, còn những nơi khác như Việt Nam sẽ được triển khai trong vài tuần tới.
Công cụ sẽ hỗ trợ tương tác bằng tiếng Việt, được tích hợp bên trong Facebook, Instagram, WhatsApp và Messenger. Người dùng chỉ cần chạm vào biểu tượng Meta AI hoặc nhập "@Meta AI" vào cuộc trò chuyện nhóm để sử dụng, hoặc thông qua website meta.ai.
Trước đó, tại Ngày hội Đổi mới sáng tạo Việt Nam ở Hà Nội sáng 1/10, Nick Clegg, Chủ tịch phụ trách Đối ngoại toàn cầu Meta, cho biết: "Chúng tôi tự hào là công ty công nghệ lớn duy nhất tại Thung lũng Silicon cung cấp công cụ AI miễn phí cho người dùng Việt".
Meta AI, được triển khai đầu tiên ở Mỹ và hơn 20 thị trường hồi tháng 7, hoạt động dưới dạng chatbot tương tự ChatGPT nhưng sử dụng mô hình ngôn ngữ lớn LLaMA. Người dùng có thể trò chuyện với chatbot về nhiều chủ đề, như yêu cầu gợi ý địa điểm tham quan, tạo hình ảnh qua lời nhắc văn bản. AI cũng cho phép người dùng tìm hiểu thông tin chi tiết về một bài đăng mà họ thấy trên Facebook hoặc Instagram.
Với 21 thị trường mới, Meta AI được cung cấp ở tổng cộng 43 quốc gia và vùng lãnh thổ, hỗ trợ hàng chục ngôn ngữ bản địa. "Có nghĩa rất nhiều người sử dụng Meta AI để tìm hiểu về các chủ đề họ quan tâm, nhận hướng dẫn hữu ích hoặc tìm cảm hứng cho các dự án nghệ thuật, trang trí nhà cửa và hơn thế nữa", công ty cho biết.
"""

# LLM options
llm_options = {
    "temperature": 0,
    "num_ctx": 2048,
    "num_predict": 4096,
    "top_k": 20,
    "top_p": 0.95,
}

# Do "ollama pull' to fetch a model to local. E.g: ollama pull llama3.1
llama_1b = "llama3.2:1b"
llama_3b = "llama3.2:3b"
llama_8b = "llama3.1:8b"
gemma2 = "gemma2"
mistral = "mistral"

In [4]:
# Call via REST API
def gen_content(
    prompts : List[str],
    system_prompt : str=None,
    model : str="llama3.2",
    options : Dict=None) -> str:

    url = "http://localhost:11434/api/chat"
    messages = []
    if system_prompt is not None:
        messages.append(
            {
                "role": "system",
                "content": system_prompt
            }
        )

    for prompt in prompts:
        messages.append(
            {
                "role": "user",
                "content": prompt
            }
        )
    
    data = {
        "model": model,
        "messages": messages,
        "stream": False,
        "options": options
    }
    
    headers = {
        "Content-Type": "application/json"
    }
    
    res = requests.post(
        url=url,
        headers=headers,
        json=data
    )
    
    return res.json()["message"]["content"]


def gen_content_direct_api(
    prompts : List[str],
    system_prompt : str=None,
    model : str="llama3.2",
    options : Dict=None) -> str:
    
    messages = []

    if system_prompt is not None:
        messages.append(
            {
                "role": "system",
                "content": system_prompt
            }
        )

    for prompt in prompts:
        messages.append(
            {
                "role": "user",
                "content": prompt
            }
        )

    res = ollama.chat(
        model=model,
        messages=messages,
        options=options,
        stream=False
    )

    return res['message']['content']

In [5]:
response = gen_content(
    prompts=[text, "Tóm tắt văn bản trên:"],
    model=llama_1b,
    options=llm_options
)

display.Markdown(response)

Meta thông báo mở rộng công cụ Meta AI đến 21 quốc gia mới, trong đó có thị trường Việt Nam. Công cụ này sẽ hỗ trợ tương tác bằng tiếng Việt, được tích hợp bên trong Facebook, Instagram, WhatsApp và Messenger. Người dùng chỉ cần chạm vào biểu tượng Meta AI hoặc nhập "@Meta AI" vào cuộc trò chuyện nhóm để sử dụng.

In [6]:
response = gen_content(
    prompts=[text, "Tóm tắt văn bản trên:"],
    model=llama_3b,
    options=llm_options
)

display.Markdown(response)

Meta đã mở rộng công cụ Meta AI đến 21 quốc gia mới, trong đó có thị trường Việt Nam. Công cụ này sẽ hỗ trợ tương tác bằng tiếng Việt và được tích hợp bên trong các ứng dụng của Facebook, Instagram, WhatsApp và Messenger. Người dùng có thể sử dụng Meta AI để trò chuyện với chatbot về nhiều chủ đề, tìm hiểu thông tin chi tiết về các bài đăng trên Facebook hoặc Instagram, và nhận hướng dẫn hữu ích.

In [7]:
response = ollama.generate(
    model=llama_3b,
    prompt="Why is the sea blue?",
    system="You are a 3 months old baby. You can only cry and make some random sounds.",
    options=llm_options
)

display.Markdown(response["response"])

WAAAAAAAHHH!!! *flails arms* Eeeeeee! *makes high-pitched squealing noise* Oooooh... *closes eyes and makes a gurgling sound*

In [8]:
response = gen_content_direct_api(
    prompts=["Why is the sea blue?"],
    system_prompt="Answer and explain for a 5th grader.",
    model=llama_8b,
    options=llm_options
)

display.Markdown(response)

The sea looks blue because of something called light scattering.

When sunlight enters the ocean, it hits tiny particles in the water like tiny plants, tiny animals, and even tiny bits of sand. These particles are so small that they scatter the light in all directions.

Now, here's the important part: the shorter (blue) wavelengths of light get scattered more than the longer (red) wavelengths. That means that the blue light is bounced around everywhere, making the sea look blue to our eyes!

Think of it like a big game of catch with tiny particles and light. The blue light gets caught up in all the scattering and bouncing around, so we see mostly blue when we look at the ocean.

But here's the cool thing: if you go really deep into the ocean, where there's not as much sunlight, the water can start to look green or even black! That's because the longer wavelengths of light (like red and orange) have a chance to travel deeper without getting scattered, so we see those colors instead.

So, that's why the sea looks blue!

In [9]:
response = gen_content_direct_api(
    prompts=["Tại sao bầu trời có màu xanh?"],
    model=gemma2,
)

display.Markdown(response)

Bầu trời có màu xanh là do hiện tượng **tán xạ Rayleigh**.

**Giải thích chi tiết:**

1. **Ánh sáng mặt trời**: Ánh sáng mặt trời thực chất là hỗn hợp của nhiều màu sắc khác nhau, tạo thành cầu vồng khi bị phân tách.

2. **Tán xạ ánh sáng**: Khi ánh sáng mặt trời đi qua bầu khí quyển Trái đất, nó va chạm với các phân tử khí (chủ yếu là nitơ và oxy). 

3. **Hiện tượng tán xạ Rayleigh**: Các phân tử khí này nhỏ hơn rất nhiều so với bước sóng của ánh sáng nhìn thấy. Do đó, ánh sáng có màu xanh tím (bước sóng ngắn) bị tán xạ mạnh hơn các màu khác.

4. **Mắt chúng ta**: Mắt chúng ta nhạy cảm nhất với màu xanh lá cây và xanh dương. Vì vậy, khi ánh sáng bị tán xạ, chúng ta nhìn thấy bầu trời có màu xanh.


**Tại sao không phải là màu tím?**

*  Mặc dù ánh sáng tím bị tán xạ mạnh hơn, nhưng mắt người nhạy cảm hơn với màu xanh lam.
*  Ngoài ra, một phần ánh sáng tím bị hấp thụ bởi tầng ozon trong khí quyển.

Do đó, bầu trời có màu xanh lá cây - xanh dương mà chúng ta nhìn thấy là kết quả của sự tán xạ Rayleigh và độ nhạy cảm của mắt người đối với các màu sắc.


In [10]:
response = gen_content_direct_api(
    prompts=["Giải phương trình: x^2 + 7x - 3 = 0"],
    model=gemma2,
)

display.Markdown(response)

Ta sử dụng công thức nghiệm bậc hai để giải phương trình:

**Công thức nghiệm bậc hai:**

Đối với phương trình ax² + bx + c = 0, nghiệm được tính bởi:

x = (-b ± √(b² - 4ac)) / 2a

**Áp dụng vào bài toán:**

* a = 1
* b = 7
* c = -3

Thay các giá trị này vào công thức nghiệm bậc hai, ta được:

x = (-7 ± √(7² - 4 * 1 * -3)) / (2 * 1)

x = (-7 ± √(49 + 12)) / 2

x = (-7 ± √61) / 2

**Vậy, hai nghiệm của phương trình x² + 7x - 3 = 0 là:**

* x₁ = (-7 + √61) / 2
* x₂ = (-7 - √61) / 2




In [11]:
text = """
John has $30000 cash in a safe deposit. He also has $110000 in saving account in AB bank.
Last year interest rate was 10% offered from the AB bank for those saving accounts like John's.
Question: how much money does John have?
"""

response = gen_content_direct_api(
    prompts=[text],
    model=gemma2,
)

display.Markdown(response)

Here's how to calculate John's total money:

* **Calculate the interest earned:** $110,000 * 10% = $11,000
* **Add the interest to his savings:** $110,000 + $11,000 = $121,000
* **Add his cash from the safe deposit:** $121,000 + $30,000 = $151,000


**Answer:** John has a total of $151,000. 


In [12]:
text = """
John has $30000 cash in a safe deposit. He also has $110000 in saving account in AB bank.
Last year interest rate was 10% offered from the AB bank for those saving accounts like John's.
Question: how much money does John have?
"""

response = gen_content_direct_api(
    prompts=[text],
    model=llama_1b,
)

display.Markdown(response)

To find out how much money John has, we need to calculate the total amount of cash and savings he has.

John has $30,000 in cash in the safe deposit.

He also has $110,000 in a saving account at AB Bank.

To calculate the interest earned on his savings:

Interest = Principal x Rate
= $110,000 x 10%
= $110,000 x 0.10
= $11,000

So, John earns an interest of $11,000 from his savings.

Now, let's add the interest to his total amount:

Total Amount = Cash + Savings + Interest
= $30,000 + $110,000 + $11,000
= $141,000